In [1]:
#Connect to GIS
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
import requests
import pandas as pd
import zipfile
import os
import io
import arcpy
from arcgis import features, GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from arcgis.features.analysis import join_features
from arcgis.features import FeatureLayerCollection
from arcgis.geometry import SpatialReference , Geometry
import json

In [3]:
directory= 'C:\Mac\Home\Documents\GIS5571_FinalProjectData'
output_gdb = r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb" 

#Get TIGER gdb
url= r'https://www2.census.gov/geo/tiger/TGRGDB24/tlgdb_2024_a_19_ia.gdb.zip'
request= requests.get(url)
with open('C:\Mac\Home\Documents\GIS5571_FinalProjectData\tlgdb_2024_a_19_ia.gdb.zip', 'wb') as file:
    file.write(request.content)

In [4]:
#unzip
zip_path ='C:\Mac\Home\Documents\GIS5571_FinalProjectData\tlgdb_2024_a_19_ia.gdb.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(directory)

In [5]:
county= r'C:\Mac\Home\Documents\GIS5571_FinalProjectData\tlgdb_2024_a_19_ia.gdb\County'

In [6]:
arcpy.conversion.FeatureClassToFeatureClass(county, output_gdb, 'county_boundaries')

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\county_boundaries'>

In [7]:
output_fc = os.path.join(output_gdb, "Electrical_Service_Boundaries")
spatial_ref = arcpy.SpatialReference(4269)

In [8]:
arcpy.management.CreateFeatureclass(output_gdb, "Electrical_Service_Boundaries", "POLYGON", spatial_reference=spatial_ref)
arcpy.management.AddField(output_fc, "Owner", "TEXT")

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\Electrical_Service_Boundaries'>

In [10]:
esb_url= r"https://services.arcgis.com/vPD5PVLI6sfkZ5E4/ArcGIS/rest/services/Electrical_Service_Boundaries/FeatureServer/14/query?where=1%3D1&&inSR=26915&f=pgeojson&token="
esbrequest= requests.get(esb_url)
response=esbrequest.json()
df=pd.json_normalize(response['features'])

In [11]:
#fixing geometry
with arcpy.da.InsertCursor(output_fc, ['SHAPE@', 'Owner']) as cursor:
    for index, row in df.iterrows():
        geom_type = row['geometry.type']
        coords = row['geometry.coordinates']
        owner = row['properties.Owner']
        
        if geom_type == 'Polygon':
            rings = coords[0]
            polygon = arcpy.Polygon(arcpy.Array([arcpy.Point(*pt) for pt in rings]), spatial_ref)
            cursor.insertRow([polygon, owner])
        
        elif geom_type == 'MultiPolygon':
            parts = []
            for part in coords:
                rings = part[0] 
                parts.append(arcpy.Array([arcpy.Point(*pt) for pt in rings]))
            multipolygon = arcpy.Polygon(arcpy.Array(parts), spatial_ref)
            cursor.insertRow([multipolygon, owner])

In [66]:
url= 'https://arcgis.netl.doe.gov/server/rest/services/Hosted/2024_Coal_Closure_Energy_Communities/FeatureServer/0/query?where=1%3D1+AND+ObjectID%3C2000+AND+ObjectID=2000&inSR=4269&f=geojson'
request=requests.get(url)
response=request.json()
df_first2000=pd.json_normalize(response['features'])

In [67]:
url= 'https://arcgis.netl.doe.gov/server/rest/services/Hosted/2024_Coal_Closure_Energy_Communities/FeatureServer/0/query?where=1%3D1+AND+ObjectID>2000&inSR=4269&f=geojson'
request=requests.get(url)
response=request.json()
df_last2000=pd.json_normalize(response['features'])

In [70]:
ec_df=pd.concat([df_first2000, df_last2000], ignore_index=True)
ec_df.head()

,type,geometry.type,geometry.coordinates,properties.state_name
0,Feature,Polygon,"[[[-80.40321037019542, 39.4375497586401], [-80...",West Virginia
1,Feature,Polygon,"[[[-80.31354933876227, 39.380858751365096], [-...",West Virginia
2,Feature,Polygon,"[[[-80.34251734767977, 39.381494750270846], [-...",West Virginia
3,Feature,Polygon,"[[[-80.7130855524263, 40.62582397306089], [-80...",Ohio
4,Feature,Polygon,"[[[-81.59651377228835, 39.95262780615861], [-8...",Ohio


In [71]:
def df_to_geojson(df, filename="EC2024.geojson"):
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }


    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": row['geometry.type'],
                "coordinates": row['geometry.coordinates']  
            },
            "properties": row.drop(['geometry.type', 'geometry.coordinates']).to_dict()
        }
        geojson["features"].append(feature)

    # Write to GeoJSON file
    with open(filename, 'w') as f:
        json.dump(geojson, f)

# Call function to save as GeoJSON
df_to_geojson(ec_df, 'C:\Mac\Home\Documents\GIS5571_FinalProjectData\EC2024.geojson')


In [72]:
arcpy.conversion.JSONToFeatures(
    in_json_file=r"C:\Mac\Home\Documents\GIS5571_FinalProjectData\EC2024.geojson",
    out_features=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\EC2024_All",
    geometry_type="POLYGON"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\EC2024_All'>

In [16]:
utility= arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="Electrical_Service_Boundaries",
    selection_type="NEW_SELECTION",
    where_clause="Owner = 'MidAmerican Energy' Or Owner = 'Interstate Power and Light Company'",
    invert_where_clause=None
)

In [18]:
output_fc = os.path.join(output_gdb, "Utility")
utility=arcpy.CopyFeatures_management(selection, output_fc)

In [19]:
arcpy.analysis.Intersect(
    in_features="Utility #;county_boundaries #",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\Utility_AHJ",
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\Utility_AHJ'>